# Classificação de Documentos

A clssificação de documentos é muito útil em vários aspectos. Um dos tipos de classificação de texto é a análise de sentimentos.

A fim de ilustrar a classificação de documentos iremos criar um modelo para classificar uma frase como positiva ou negativa.

## Carregando o embedding e os dados

In [1]:
!pip install unidecode
!pip install vaderSentiment

In [39]:
import gensim
import pandas as pd
from nltk.corpus import stopwords
import string
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import re
import numpy as np

In [66]:
#opção 1 -> montar o drive no colab e acessar o arquivo de embedding do drive
from google.colab import drive
drive.mount('/content/drive')

#opção 2 -> fazer download e fazer upload por aqui
#from google.colab import files
#uploaded = files.upload()

In [22]:
%%time
path='drive/MyDrive/aulas/Processamento de Linguagem Natural - Saude/ptwiki_20180420_100d.txt.bz2'
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(path,
                                                               binary=False,
                                                               limit=50000)

In [69]:
df = pd.read_csv('drive/MyDrive/aulas/Processamento de Linguagem Natural - Saude/imdb-reviews-pt-br.csv')

In [71]:
df.sentiment.value_counts()

## Tratamento dos dados

1. Transforme a variavel alvo (sentiment) em uma variável binaria
2. Faço o seguinte pré-processamento no texto:
  1. tokenize as expressões usando regex "\w+(?:'\w+)?|[^\w\s]"
  2. Passe tudo para minusculo
  3. Remova stopwords
  4. Remova pontuação
  5. Remova números

  Faça duas funções de pré-processamento uma que retorne a frase processada e uma que retorne uma lista com os tokens.

## Solução

In [23]:
df = pd.read_csv('drive/MyDrive/aulas/Processamento de Linguagem Natural - Saude/imdb-reviews-pt-br.csv')

In [24]:
target = df['sentiment'].replace(['neg','pos'],[0,1])

In [25]:
def pre_processamento_texto_return_str(corpus, portugues_stops):
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus) # extração palavras -> palavras
    corpus_alt = [t.lower() for t in corpus_alt] #passando para minusculo
    corpus_alt = [t for t in corpus_alt if t not in portugues_stops] #remoção dos tokens que são SW
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation] # o mesmo para pontuação
    corpus_alt = [re.sub(r'\d', '', t) for t in corpus_alt] #remoção de numeros
    corpus_alt_str = ' '.join(corpus_alt)
    return corpus_alt_str

In [26]:
def pre_processamento_texto_return_token(corpus, portugues_stops):
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    corpus_alt = [t.lower() for t in corpus_alt]
    portugues_stops = stopwords.words('portuguese')
    corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]
    corpus_alt = [re.sub(r'\d', '', t) for t in corpus_alt]

    return corpus_alt

In [27]:
import nltk
nltk.download('stopwords')

In [28]:
portugues_stops = stopwords.words('portuguese')

In [31]:
from tqdm import tqdm
tqdm.pandas()

In [73]:
df["text_pt_sem_stopwords"] = df["text_pt"]\
      .progress_apply(lambda x: pre_processamento_texto_return_str(x, portugues_stops))

In [74]:
df["text_pt_sem_stopwords"]

## Bag-of-word

Crie uma representação bag-of-words do texto.

## Solução

In [75]:
vect_bag = CountVectorizer()
X_bag = vect_bag.fit_transform(df['text_pt_sem_stopwords'])
vocabulario = vect_bag.get_feature_names_out()

In [76]:
print("Vocabulario",len(vocabulario))
print("Features",X_bag.shape)
print("Target",target.shape)

## Embedding

Utilizando o embedding crie uma representação de embedding com a média das representações das palavras do texto.

## Solução

In [36]:
df["text_pt_sem_stopwords_token"] = df["text_pt"]\
.progress_apply(lambda x: pre_processamento_texto_return_token(x, portugues_stops))

In [50]:
def calcula_embedding_frase(tokens):
    return np.mean([word_vectors[t] for t in tokens if t in word_vectors.key_to_index.keys()], axis=0)

In [51]:
X_embedding = df["text_pt_sem_stopwords_token"].progress_apply(lambda x: calcula_embedding_frase(x))

In [77]:
X_embedding

### Treinamento

Separe o banco de bag of words em treino e teste e treine um modelo de regressão Logistica, qual a avaliação desse modelo?

## Solução

In [78]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = \
    train_test_split(X_bag, target, random_state=123)

In [53]:
modelo_bow = LogisticRegression()
modelo_bow.fit(X_train_bow,y_train_bow)

In [54]:
y_pred = modelo_bow.predict(X_test_bow)

In [55]:
print(classification_report(y_test_bow, y_pred))

### Embedding

Separe o banco de embedding em treino e teste e treine um modelo de regressão Logistica, qual a avaliação desse modelo?

## Solução

In [56]:
X_train_embedding, X_test_embedding, y_train_embedding, y_test_embedding = \
train_test_split(X_embedding.values, target,random_state=123)

In [57]:
X_train_embedding = pd.DataFrame([x for x in X_train_embedding])
X_test_embedding = pd.DataFrame([x for x in X_test_embedding])

In [58]:
modelo_embedding = LogisticRegression()
modelo_embedding.fit(X_train_embedding,y_train_embedding)

In [59]:
y_pred = modelo_embedding.predict(X_test_embedding)

In [60]:
print(classification_report(y_test_embedding, y_pred))

# Análise de sentimentos

Quando o objetivo é realizar análise de sentimentos podemos treinar o nosso proprio modelo ou utilizar ferramentas já feitas. Exemplo: Vader.

O VADER (Valence Aware Dictionary e sEntiment Reasoner) é uma ferramenta de análise de sentimentos baseada em regras e léxico, especificamente identifica os sentimentos expressos nas mídias sociais.

- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) e (compound score < 0.05)
- negative sentiment: compound score <= -0.05

Mais informações: https://github.com/cjhutto/vaderSentiment

In [79]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [80]:
analyzer = SentimentIntensityAnalyzer()

In [81]:
texto_neg = df.loc[0, "text_en"]
texto_pos = df.loc[49431, "text_en"]

In [82]:
texto_neg

In [64]:
analyzer.polarity_scores(texto_neg)

In [83]:
analyzer.polarity_scores(texto_pos)